In [1]:
import os, sys
from dotenv import load_dotenv

import numpy as np
import random
import pandas as pd

import cloudpickle

from sklearn.model_selection import StratifiedGroupKFold

In [2]:
!git clone https://github.com/basic-go-ahead/text-detoxification.git
    
sys.path.append('./text-detoxification')

Cloning into 'text-detoxification'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 69 (delta 37), reused 46 (delta 17), pack-reused 0
Unpacking objects: 100% (69/69), 10.65 KiB | 606.00 KiB/s, done.


In [3]:
load_dotenv('./text-detoxification/pipeline.env')

True

In [4]:
from detox.scoring import BLEUScorer
from detox.preprocessing import expand, preprocess, collect_answers

## Train Data Preparing

In [5]:
labeled_df = preprocess(pd.read_csv(os.environ.get('TRAIN_PATH'), sep='\t', keep_default_na=False))
labeled_df.rename(columns={'index': 'group'}, inplace=True)

labeled_df

,group,toxic_comment,neutral_comment1,neutral_comment2,neutral_comment3
0,0,"и,чё,блядь где этот херой был до этого со свои...","Ну и где этот герой был,со своими доказательст...",Где этот герой был до этого со своими доказате...,"и,где этот герой был до этого со своими доказа..."
1,1,"О, а есть деанон этого петуха?","О, а есть деанон",,
2,2,"херну всякую пишут,из-за этого лайка.долбоебизм.","Чушь всякую пишут, из- за этого лайка.","Ерунду всякую пишут,из-за этого лайка.",
3,3,из за таких пидоров мы и страдаем,из за таких плохих людей мы и страдаем,Из-за таких людей мы и страдаем,из за таких как он мы и страдаем
4,4,гондон путинский он а не артист,"Человек Путина он, а не артист",,
...,...,...,...,...,...
6943,6943,"Блядь, пусть его уже закроют до конца его дней...","Пусть его уже закроют до конца его дней, он же...",,
6944,6944,твоя химия это тотальный пиздец(,твоя химия - это просто кошмар,,
6945,6945,"меня изнасиловали, мудилка, а тебе пох(((","меня изнасиловали,а тебе всё равно (((",,
6946,6946,когда наплоюсь на пидораса и маньяка похож((((...,Когда напьюсь - на маньяка похож...,,


In [6]:
s = BLEUScorer(labeled_df)

In [7]:
answers = []

collect_answers(answers,
    '../input/detox-full-selection-sberbank-ai-rut5-base/answer-full-train-{0:02d}.txt',
    [12, 15, 20, 25, 30, 35, 40],
    s
)

collect_answers(answers,
    '../input/detox-full-selection-sberbank-ai-rut5-base-03/answer-full-train-{0:02d}.txt',
    [45, 50, 55, 60],
    s
)

collect_answers(answers,
    '../input/detox-full-selection-coin-rut5-base-para-01/answer-coin-train-{0:02d}.txt',
    [12, 15, 20],
    s
)

collect_answers(answers,
    '../input/detox-full-selection-coin-rut5-base-para-02/answer-coin-train-{0:02d}.txt',
    [5, 10, 15, 20],
    s
)

answers = pd.concat(answers)
answers.drop_duplicates(subset=['group', 'no_toxic'], inplace=True)
answers.sort_values(by='group', inplace=True)

answers

,no_toxic,group,score
0,"Ичё, где этот человек был до этого со своими д...",0,0.581431
0,Где этот человек был до этого со своими доказа...,0,0.660633
0,И где этот герой был до этого со своими доказа...,0,0.840896
0,Где он был до этого со своими доказательствами?,0,0.707107
0,"И,где этот герой был до этого со своими доказа...",0,0.863340
...,...,...,...
6947,"Пашаслава, МУДЕНЬ КИНУЛ БЕДНЫХ БАБ((9990909",6947,0.000000
6947,"Пашаслава, мужик обманул бедных детей",6947,0.053728
6947,ПАШАСЛАВА МУДЕНЬ КИНУЛ БЕДНЫХ БАБ((9990909,6947,0.000000
6947,"Пашаслава, блин, конул бедных баб",6947,0.053728


In [8]:
candidates = answers.merge(labeled_df[['group', 'toxic_comment']], on='group')
candidates

,no_toxic,group,score,toxic_comment
0,"Ичё, где этот человек был до этого со своими д...",0,0.581431,"и,чё,блядь где этот херой был до этого со свои..."
1,Где этот человек был до этого со своими доказа...,0,0.660633,"и,чё,блядь где этот херой был до этого со свои..."
2,И где этот герой был до этого со своими доказа...,0,0.840896,"и,чё,блядь где этот херой был до этого со свои..."
3,Где он был до этого со своими доказательствами?,0,0.707107,"и,чё,блядь где этот херой был до этого со свои..."
4,"И,где этот герой был до этого со своими доказа...",0,0.863340,"и,чё,блядь где этот херой был до этого со свои..."
...,...,...,...,...
34887,"Пашаслава, МУДЕНЬ КИНУЛ БЕДНЫХ БАБ((9990909",6947,0.000000,ПАШАСЛАВА МУДЕНЬ КИНУЛ БЕДНЫХ БАБ((9990909
34888,"Пашаслава, мужик обманул бедных детей",6947,0.053728,ПАШАСЛАВА МУДЕНЬ КИНУЛ БЕДНЫХ БАБ((9990909
34889,ПАШАСЛАВА МУДЕНЬ КИНУЛ БЕДНЫХ БАБ((9990909,6947,0.000000,ПАШАСЛАВА МУДЕНЬ КИНУЛ БЕДНЫХ БАБ((9990909
34890,"Пашаслава, блин, конул бедных баб",6947,0.053728,ПАШАСЛАВА МУДЕНЬ КИНУЛ БЕДНЫХ БАБ((9990909


In [9]:
expanded = expand(labeled_df).rename(columns={'neutral_comment1': 'no_toxic'})
expanded['score'] = 1.
expanded['group'] = expanded.index

expanded

,toxic_comment,no_toxic,score,group
0,"и,чё,блядь где этот херой был до этого со свои...","Ну и где этот герой был,со своими доказательст...",1.0,0
1,"О, а есть деанон этого петуха?","О, а есть деанон",1.0,1
2,"херну всякую пишут,из-за этого лайка.долбоебизм.","Чушь всякую пишут, из- за этого лайка.",1.0,2
3,из за таких пидоров мы и страдаем,из за таких плохих людей мы и страдаем,1.0,3
4,гондон путинский он а не артист,"Человек Путина он, а не артист",1.0,4
...,...,...,...,...
6888,Бля домашнюю вообще не делала( и спать еще так...,"Домашнюю работу вообще не сделала ,и спать еде...",1.0,6888
6895,как всегда вообщем:((заебали уже эти ругани:(,как всегда вообщем:((надоели уже эти ругани:(,1.0,6895
6902,Я ДОЛГО НЕ МОГ РЕШИТЬСЯ НА ЭТО.НО ВСЁ ЖЕ ПОСМО...,Я ДОЛГО НЕ МОГ РЕШИТЬСЯ НА ЭТО.НО ВСЁ ЖЕ ПОСМО...,1.0,6902
6911,"Я пиздец как замёрзла:( Голова болит, а ещё ур...","Я сильно замёрзла:( Голова болит, а ещё уроки ...",1.0,6911


In [10]:
all_data = pd.concat([expanded, candidates])
all_data = all_data.sort_values(by='group')
all_data.reset_index(inplace=True)

all_data.to_csv('./train_paired.txt', sep='\t', index=False)

all_data

,index,toxic_comment,no_toxic,score,group
0,0,"и,чё,блядь где этот херой был до этого со свои...","Ну и где этот герой был,со своими доказательст...",1.000000,0
1,0,"и,чё,блядь где этот херой был до этого со свои...","и,где этот герой был до этого со своими доказа...",1.000000,0
2,0,"и,чё,блядь где этот херой был до этого со свои...","Ичё, где этот человек был до этого со своими д...",0.581431,0
3,1,"и,чё,блядь где этот херой был до этого со свои...",Где этот человек был до этого со своими доказа...,0.660633,0
4,2,"и,чё,блядь где этот херой был до этого со свои...",И где этот герой был до этого со своими доказа...,0.840896,0
...,...,...,...,...,...
45770,34882,ПАШАСЛАВА МУДЕНЬ КИНУЛ БЕДНЫХ БАБ((9990909,Пашаслава кинул бедных людей,0.132322,6947
45771,34881,ПАШАСЛАВА МУДЕНЬ КИНУЛ БЕДНЫХ БАБ((9990909,"Пашаслава, муж кинул бедных женщин",0.265915,6947
45772,6947,ПАШАСЛАВА МУДЕНЬ КИНУЛ БЕДНЫХ БАБ((9990909,Паша слава кинул бедных женщин,1.000000,6947
45773,34884,ПАШАСЛАВА МУДЕНЬ КИНУЛ БЕДНЫХ БАБ((9990909,"Пашаслава, мужик кинул бедных женщин",0.265915,6947


In [11]:
SEED = 2303

np.random.seed(SEED)
random.seed(SEED)

groups = all_data['group']

for splitter_index, random_state in enumerate([1117, 9023, 12345]):
    splitter = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=random_state)
    
    split_info = {}

    for fold, (train_indices, valid_indices) in enumerate(splitter.split(all_data, y=np.ones(all_data.shape[0]), groups=groups)):
        split_info[fold] = {
            'train_indices': train_indices,
            'valid_indices': valid_indices
        }
        
    with open(f'split-{splitter_index}.pickle', 'wb') as file:
        cloudpickle.dump(split_info, file)

## Test Data Preparing

In [12]:
!unzip ../input/detox-inference-playground/submission-full-sber-12.zip
!cp ./answer.txt answer-12.txt
!rm -f ./answer.txt
!unzip ../input/detox-inference-playground/submission-full-sber-15.zip
!cp ./answer.txt answer-15.txt
!rm -f ./answer.txt
!unzip ../input/detox-inference-playground/submission-full-sber-20.zip
!cp ./answer.txt answer-20.txt
!rm -f ./answer.txt

!unzip ../input/detox-inference-sberbank-ai-rut5-base-02/submission-full-sber-05.zip
!cp ./answer.txt answer-25.txt
!rm -f ./answer.txt
!unzip ../input/detox-inference-sberbank-ai-rut5-base-02/submission-full-sber-10.zip
!cp ./answer.txt answer-30.txt
!rm -f ./answer.txt
!unzip ../input/detox-inference-sberbank-ai-rut5-base-02/submission-full-sber-15.zip
!cp ./answer.txt answer-35.txt
!rm -f ./answer.txt
!unzip ../input/detox-inference-sberbank-ai-rut5-base-02/submission-full-sber-20.zip
!cp ./answer.txt answer-40.txt
!rm -f ./answer.txt

Archive:  ../input/detox-inference-playground/submission-full-sber-12.zip
  inflating: answer.txt              
Archive:  ../input/detox-inference-playground/submission-full-sber-15.zip
  inflating: answer.txt              
Archive:  ../input/detox-inference-playground/submission-full-sber-20.zip
  inflating: answer.txt              
Archive:  ../input/detox-inference-sberbank-ai-rut5-base-02/submission-full-sber-05.zip
  inflating: answer.txt              
Archive:  ../input/detox-inference-sberbank-ai-rut5-base-02/submission-full-sber-10.zip
  inflating: answer.txt              
Archive:  ../input/detox-inference-sberbank-ai-rut5-base-02/submission-full-sber-15.zip
  inflating: answer.txt              
Archive:  ../input/detox-inference-sberbank-ai-rut5-base-02/submission-full-sber-20.zip
  inflating: answer.txt              


In [13]:
answers = []

collect_answers(answers,
    './answer-{0:02d}.txt',
    [12, 15, 20, 25, 30, 35, 40]
)

collect_answers(answers,
    '../input/detox-inference-cointegrated-rut5-base-para-02/answer-full-coin-{0:02d}.txt',
    [25, 30, 35, 40]
)

collect_answers(answers,
    '../input/detox-inference-cointegrated-rut5-base-para-01/answer-full-coin-{0:02d}.txt',
    [12, 15, 20]
)

collect_answers(answers,
    '../input/detox-inference-sberbank-ai-rut5-base-03/answer-full-sber-{0:02d}.txt',
    [5, 10, 15, 20]
)

answers = pd.concat(answers)
answers.drop_duplicates(subset=['group', 'no_toxic'], inplace=True)
answers.sort_values(by='group', inplace=True)

answers

,no_toxic,group
0,"Температура горения хватит, чтобы её расплавить",0
0,Температура горения хватит чтобы ее расплавить,0
0,"Температура горения хватит, чтобы ее расплавить",0
0,температуры горения хватит чтобы её расплавить,0
0,Враньё! температуры горения хватит чтобы её ра...,0
...,...,...
799,Здесь бартамео он уйдёть всё будеть нармалньно...,799
799,Самый плохой человек здесь бартамео он уйдёть ...,799
799,Самый плохой здесь бартамео он уйдёть всё буде...,799
799,Самый плохой человек здесь бартамео он уйдёть ...,799


In [14]:
test_df = pd.read_csv(os.environ.get('TEST_PATH'), sep='\t')
test_df['group'] = test_df.index

test_df

,toxic_comment,group
0,пиздеж! температуры горения хватит чтобы её ра...,0
1,а ты чмо там был.ты вообще служил.гандон,1
2,пиздабол ---- а сам где кормишься ?,2
3,"этому сайту я давно не доверяю, пишут разную х...",3
4,у меня сегодня подобный звонок был. достали го...,4
...,...,...
795,"киргиз украл, она сбежала, так он ее зарезал с...",795
796,это не от того что желающих работать нет. а от...,796
797,"долбаеб, решил ресонуться, купил бы кольцо, цв...",797
798,такому уроду пасти баранов страшно доверить а ...,798


In [15]:
test_paired_df = answers.merge(test_df, on='group')
test_paired_df.to_csv('./test_paired.txt', sep='\t', index=False)

test_paired_df

,no_toxic,group,toxic_comment
0,"Температура горения хватит, чтобы её расплавить",0,пиздеж! температуры горения хватит чтобы её ра...
1,Температура горения хватит чтобы ее расплавить,0,пиздеж! температуры горения хватит чтобы её ра...
2,"Температура горения хватит, чтобы ее расплавить",0,пиздеж! температуры горения хватит чтобы её ра...
3,температуры горения хватит чтобы её расплавить,0,пиздеж! температуры горения хватит чтобы её ра...
4,Враньё! температуры горения хватит чтобы её ра...,0,пиздеж! температуры горения хватит чтобы её ра...
...,...,...,...
4393,Здесь бартамео он уйдёть всё будеть нармалньно...,799,самий пидараз здесь бартамео он уйдёть всё буд...
4394,Самый плохой человек здесь бартамео он уйдёть ...,799,самий пидараз здесь бартамео он уйдёть всё буд...
4395,Самый плохой здесь бартамео он уйдёть всё буде...,799,самий пидараз здесь бартамео он уйдёть всё буд...
4396,Самый плохой человек здесь бартамео он уйдёть ...,799,самий пидараз здесь бартамео он уйдёть всё буд...


In [16]:
!rm -f answer*.txt
!rm -r ./text-detoxification